# PyMem3DG Tutorial 7 - Unwrapped integrator

It is possible to write an integrator directly in Python such that there is more control over its behavior. It could also be useful if you want to modify certain parameters/data during simulation. Here is a snippet on how to write a simple forward Euler integrator. 

In [2]:
import pymem3dg as dg
oblate = "sample_meshes/oblate.ply"
p = dg.Parameters()
p.bending.Kbc = 8.22e-5
p.tension.Ksg = 0.1
p.tension.At = 12.4866
p.osmotic.isPreferredVolume = True
p.osmotic.Kv = 0.02
p.osmotic.Vt = 0.7 * 3.14 * 4 / 3

In [9]:
g = dg.System(inputMesh=oblate, parameters=p)
g.initialize()

outputDir = "output/tutorial7"
fe = dg.Euler(system=g, characteristicTimeStep=1,
              tolerance=1e-5, outputDirectory=outputDir)

frame = 0
lastSave = g.time
lastProcessMesh = g.time
initTime = g.time
tSave = 2000 * fe.characteristicTimeStep
processMeshPeriod = 30

try:
    fe.integrate()
except:
    print("Function deleted!\n")

fe.createMutableNetcdfFile(isContinue=False)
while frame < 10:
    # compute forces/energy, and test for fe.EXIT
    fe.status()

    # saveData
    if ((g.time - lastSave > tSave) | (g.time == initTime) | fe.EXIT):
        lastSave = g.time
        fe.saveData(ifOutputTrajFile=True,
                    ifOutputMeshFile=False, ifPrintToConsole=True)
        frame = frame + 1

    if (fe.EXIT):
        break

    # mesh processing
    if (g.time - lastProcessMesh > (processMeshPeriod * fe.timeStep)):
        lastProcessMesh = g.time
        g.mutateMesh(1)
        g.updateConfigurations()

    # time stepping and update variables
    if (g.time == lastProcessMesh):
        g.time = g.time + 1e-10 * fe.characteristicTimeStep
    else:
        fe.march()

fe.closeMutableNetcdfFile()


Function deleted!

area_init = 12.4783
vol_init = 4.02477
Characteristic volume wrt to At = 4.14897

t: 0, n: 0, isSmooth: 1
A, At: 12.4783, 12.4866, V, Vt: 4.02477, 2.93067, h: 0.80235
E_total: 0.00249136
E_kin: 0
E_pot: 0.00249136
W_ext: 0
|e|Mech: 0.00144411
|e|Chem: 0
H: [0.61458,1.48814]
K: [0.411476,1.78376]
phi: [1,1]
sum_phi: 12.4783

t: 2002.97, n: 1, isSmooth: 1
A, At: 12.355, 12.4866, V, Vt: 3.90856, 2.93067, h: 0.753121
E_total: 0.00230907
E_kin: 2.90727e-08
E_pot: 0.00230905
W_ext: 0
|e|Mech: 0.000241192
|e|Chem: 0
H: [0.591207,1.36192]
K: [0.335478,1.82495]
phi: [1,1]
sum_phi: 12.355

t: 4003.09, n: 2, isSmooth: 1
A, At: 12.3686, 12.4866, V, Vt: 3.83773, 2.93067, h: 0.700644
E_total: 0.00218316
E_kin: 3.32415e-08
E_pot: 0.00218313
W_ext: 0
|e|Mech: 0.00025786
|e|Chem: 0
H: [0.4829,1.43242]
K: [0.232379,1.9527]
phi: [1,1]
sum_phi: 12.3686

t: 6004.92, n: 3, isSmooth: 1
A, At: 12.3837, 12.4866, V, Vt: 3.7518, 2.93067, h: 0.645156
E_total: 0.00204906
E_kin: 3.29352e-08
E_pot

In [10]:
import pymem3dg.visual as dg_vis
import polyscope as ps
dg_vis.animate(outputDir+"/traj.nc", meanCurvature = True)
ps.show()
